In [2]:
# Section 1: Importing Necessary Libraries
# Importing libraries for loading the MNIST dataset and hyperparameter tuning
from tensorflow.keras.datasets import mnist
import keras_tuner as kt
from kerastuner.engine.hyperparameters import HyperParameters
from tensorflow.keras import layers, models
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense, Dropout
from a1 import build_deep_nn

In [3]:
from tensorflow.keras.datasets import mnist

# Section 2: Loading the MNIST Dataset

# Load the MNIST dataset
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

# Normalize the pixel values to the range [0, 1]
train_images = train_images.astype('float32') / 255.0
test_images = test_images.astype('float32') / 255.0

# Print the shape of the training and test sets
print("Training images shape:", train_images.shape)
print("Training labels shape:", train_labels.shape)
print("Test images shape:", test_images.shape)
print("Test labels shape:", test_labels.shape)


Training images shape: (60000, 28, 28)
Training labels shape: (60000,)
Test images shape: (10000, 28, 28)
Test labels shape: (10000,)


In [4]:
# Section 3: Defining the Hyperparameter Search Space and Tuning

# Define the search space for hyperparameters
def model_builder(hp):
    model = Sequential()
    model.add(Flatten(input_shape=(28, 28)))  # Input layer

    # Tune the number of hidden layers (1 to 3)
    hp_num_hidden = hp.Int('num_hidden', min_value=1, max_value=3)
    for i in range(hp_num_hidden):
        # Tune the size of each hidden layer (100 to 512)
        hp_hidden_size = hp.Int(f'hidden_size_{i}', min_value=100, max_value=512)
        model.add(Dense(units=hp_hidden_size, activation='relu'))
        
        # Tune the dropout rate for each hidden layer (0.0 to 0.5)
        hp_dropout_rate = hp.Float(f'dropout_rate_{i}', min_value=0.0, max_value=0.5)
        model.add(Dropout(rate=hp_dropout_rate))

    # Output layer
    model.add(Dense(units=10, activation='softmax'))

    # Compile the model
    model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    
    return model

# Initialize the tuner
tuner = kt.Hyperband(model_builder,
                     objective='val_accuracy',
                     max_epochs=10,
                     factor=3,
                     directory='keras_tuner_dir',
                     project_name='mnist_hyperparameter_tuning')

# Perform the hyperparameter search
tuner.search(train_images, train_labels,
             epochs=10,
             validation_split=0.2)

# Get the best hyperparameters
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
print("Best Hyperparameters:")
print(best_hps)


Reloading Tuner from keras_tuner_dir/mnist_hyperparameter_tuning/tuner0.json
Best Hyperparameters:


In [5]:
# Section 4: Training the Model with the Best Hyperparameters

# Build the model with the best hyperparameters
model = tuner.hypermodel.build(best_hps)

# Train the model
history = model.fit(train_images, train_labels, epochs=10, validation_split=0.2)


Epoch 1/10
  56/1500 [>.............................] - ETA: 1s - loss: 1.0598 - accuracy: 0.6858  

2024-03-08 17:51:48.639482: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


1500/1500 [==============================] - 2s 1ms/step - loss: 0.2729 - accuracy: 0.9196 - val_loss: 0.1330 - val_accuracy: 0.9613
Epoch 2/10
1500/1500 [==============================] - 2s 1ms/step - loss: 0.1251 - accuracy: 0.9625 - val_loss: 0.0960 - val_accuracy: 0.9710
Epoch 3/10
1500/1500 [==============================] - 2s 1ms/step - loss: 0.0882 - accuracy: 0.9727 - val_loss: 0.0834 - val_accuracy: 0.9737
Epoch 4/10
1500/1500 [==============================] - 2s 1ms/step - loss: 0.0729 - accuracy: 0.9767 - val_loss: 0.0831 - val_accuracy: 0.9743
Epoch 5/10
1500/1500 [==============================] - 2s 1ms/step - loss: 0.0570 - accuracy: 0.9818 - val_loss: 0.0807 - val_accuracy: 0.9779
Epoch 6/10
1500/1500 [==============================] - 2s 1ms/step - loss: 0.0486 - accuracy: 0.9839 - val_loss: 0.0806 - val_accuracy: 0.9760
Epoch 7/10
1500/1500 [==============================] - 2s 1ms/step - loss: 0.0431 - accuracy: 0.9855 - val_loss: 0.0765 - val_accuracy: 0.9788
Epo

In [7]:
# Section 5: Evaluating the Model on the Test Set

# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(test_images, test_labels)

print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)




313/313 [==============================] - 0s 457us/step - loss: 0.0659 - accuracy: 0.9822
Test Loss: 0.06587814539670944
Test Accuracy: 0.982200026512146
